In [20]:
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

##  Note: Multiple key presses within a short interval (within 4 seconds) are considered as a single attitude change, with the timing taken as the time of the first key press.

In [27]:

def filter_time_series(df, time_col='time', rate_col='rate', time_threshold=4):
    """
    合并时间接近的数据点并删除连续相同rate的后一个数据点。
    
    参数：
    df (pd.DataFrame): 输入的DataFrame，包含时间和rate列。
    time_col (str): 时间列的列名。
    rate_col (str): rate列的列名。
    time_threshold (int): 时间间隔阈值，单位为秒。

    返回值：
    pd.DataFrame: 处理后的DataFrame。
    """
    # 合并时间接近的数据点
    merged_data = []
    start_time = df[time_col][0]
    last_rate = df[rate_col][0]

    for i in range(1, len(df)):
        if df[time_col][i] - df[time_col][i-1] < time_threshold:
            last_rate = df[rate_col][i]
        else:
            merged_data.append({time_col: start_time, rate_col: last_rate})
            start_time = df[time_col][i]
            last_rate = df[rate_col][i]

    # 添加最后一个合并的数据点
    merged_data.append({time_col: start_time, rate_col: last_rate})

    # 转换为 DataFrame
    merged_df = pd.DataFrame(merged_data)

    # 删除连续相同rate的后一个数据点
    filtered_data = []
    for i in range(len(merged_df)):
        if i == 0 or merged_df[rate_col][i] != merged_df[rate_col][i-1]:
            filtered_data.append({time_col: merged_df[time_col][i], rate_col: merged_df[rate_col][i]})

    filtered_df = pd.DataFrame(filtered_data)

    return filtered_df


In [22]:
def generate_rate_data_by_TR(df):
    
    # 提取时间和评分数据
    times = df['time'].values
    rates = df['rate'].values
    
    # 生成新的时间点（每2秒一个数据点，直到3000秒）
    new_times = np.arange(0, 2987, 2)
    
    # 创建新的评分数据列表
    new_rates = []
    current_rate = rates[0]
    current_time_index = 0
    
    # 遍历新的时间点
    for t in new_times:
        # 检查是否需要更新评分
        if current_time_index < len(times) - 1 and t >= times[current_time_index + 1]:
            current_time_index += 1
            current_rate = rates[current_time_index]
        new_rates.append(current_rate)
    
    # 创建新的DataFrame
    new_df = pd.DataFrame({'time': new_times, 'rate': new_rates})
    
    return new_df

In [28]:
base_dir = '/Users/li/Desktop/task-debate/behavdata/during_scan'

sub_list = [f'{x}' for x in range(13,51)]

for sub in sub_list:
    
    file = f'/Users/li/Desktop/task-debate/behavdata/during_scan/combined_6runs/subject_{sub}_rate-time.csv'
    df = pd.read_csv(file)
    
    filter_df = filter_time_series(df)
    TR_df = generate_rate_data_by_TR(filter_df)

    
    output_file_1 = os.path.join(base_dir, 'combined_6runs_filter', f'subject_{sub}_rate-time.csv')
    filter_df.to_csv(output_file_1, index=False)
    
    output_file_2 = os.path.join(base_dir, 'combined_6runs_per_TR_filter', f'subject_{sub}_TR_rate.csv')
    TR_df.to_csv(output_file_2, index=False)
    